In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

- in our sequence of algorithms for estimating the state of a system, Particle Filters is the third one and in many ways is the best one
  - it's the easiest to program and in most ways is the most flexible


- to understand why I'm saying this let me start with a little quiz that goes back into the first 2 classes
- in class 1, we learned about Histogram Filters, in class 2 about Kalman Filters, and we even had a proof there

# State Space

- **Q:** I'd like to know whether the state space was discrete or continuous for Histogram Filters and Kalman Filters?
- **A:** When a Histogram Filter was discrete, distribution was defined over a finite set of bins, whereas the Kalman filter had a continuous state space.

# Belief Modality

- **Q:** I would like to know whether distributions that can be represented may be unimodal or can also be multimodal? So, check unimodal if this is all we can do, whereas if we can have multiple bumps in our probability distribution, check multimodal.
- **A:** Here the Histogram Filter scores better--even though it was discrete, it was able to represent multiple bumps, which the Kalman Filter couldn't--so it's unimodal. The Kalman Filter was a single Gaussian which is by definition unimodal, whereas the Histogram Filter can have bumps over arbitrary grid cells.

# Efficiency

- **Q:** The next question I wouldn't need to dwell on in the class, but I think it's important. When it comes to scaling in the number of dimensions of the state space, the amount of storage we have to assign? I give you 2 answers--it could be quadratic or exponential. I understand I didn't really discuss this, but go back in your memory to how grids are represented and how Gaussian's are represented.
- **A:** The Histogram Filter's biggest disadvantage is it scales exponentially, and the reason is any grid that is defined over $k$ dimensions will end up having exponentially many grid cells in the number of dimensions, which is really unfortunate because we can't really represent high dimensional grids really well. So, it works really well for low dimensional problems like 3 dimensional robot localization problems. The Kalman Filter in contrast, under certain assumptions, is quadratic. All we represented was a vector, the mean, and the covariance matrix, and the covariance matrix is quadratic. And it turns out all the computation, if your measurements space is a fixed size, ends up to be quadratic (without a proof here so, you just have to take it by faith). So, if you have a 15, 20 dimensional state space, the Kalman Filters will be more efficient than the Histogram Filters.

# Exact or Approximate

- **Q:** When applied to robotics do we believe the Histogram Filter and Kalman Filter are exact or approximate?
- **A:** Histogram Filters tend to be approximate because the world tends not to be discrete. So localization, for example, it's clearly an approximate filter. It turns out Kalman Filters are also approximate, and it's a much more subtle observation--it turns out Kalman Filters only are exact for linear systems, whereas the world happens to be nonlinear.
  - Now this goes into a lot of deep math, which I don't want to get into here, but you should understand that both of these filters are not exact. Both of them tend to be just approximations of the correct posterior distribution.

# Particle Filters

- now let's look into Particle Filters, the subject of today's class, and it's really interesting to see the answers for particle filters
  - the state space for particle filters is usually continuous--so, you can get into the more interesting version of state spaces
  - we're not confined to unimodal distributions--we can actually represent arbitrarily multimodal distributions
  - they are also approximate just like the other 2 filters
  - in terms of efficiency the world is still out there--in certain incarnations, they clearly scale exponentially, and it is a mistake to represent Particle Filters over anything more than say 4 dimensions
    - but in other domains, mostly in tracking domains, they tend to scale much, much better, and I've not seen a good treatment yet of the complexity in practice for Particle Filters


- the key advantage of Particle Filters is actually none of those things over here--the key advantage, at least in my life, has been they're really easy to program


- you will write your own particle filter for a continuous value localization problem, which is in many ways more difficult than any of the problems we talked about before
- https://youtu.be/4S-sx5_cmLU?t=83
  - here is a floor plan of an environment where a robot is located and it has to perform what's called global localization, which is it has no clue where it is and it has to find out where it is just based on sensor measurements
  - this provides his range sensors as indicated by the blue stripes--those use sonar sensors, which means sound, to range the distance of nearer obstacles, and it has to use these range sensors to determine a good posterior distribution as to where it is
  - what the robot doesn't know it's starting in the middle of the corridor--in fact, it is completely uncertain as to where it is
  - now, the particle filter represents this using particles--each of these red dots of which there are several thousand here is a discrete guess where the robot might be
    - it's structured as an $x$ coordinate, a $y$ coordinate, and also a heading direction, and these 3 values together comprise a single guess, but a single guess is not a filter
    - it is the set of several thousands of such guesses that together comprise an approximate representation for the posterior of the robot
  - in the beginning the particles are uniformly spread, but the particle filter makes them survive in proportion of how consistent one of these particles is with a sensor measurement
  - the essence of particle filters is to have these particles guess where the robot might be moving, but also have them survive using effectively survival of the fittest so that particles that are more consistent with the measurements are more likely to survive and as a result places of high probability will collect more particles, and therefore be more representative of the robot's posterior belief
  - those particles together--those thousands of particles are in the end clustered in a single location--those comprise the approximate belief of the robot as it localizes itself

# Using Robot Class

- Sebastian wrote a piece of code for you that I am now going to demonstrate
- the main class is a class called `robot`
- this robot lives in a 2-dimensional world of size $100x100$ meters
- it can see 4 different landmarks that are located at the following coordinates: $20, 20; 80,80; 20,80; 80,20$
- to instantiate such a robot all you have to do is call a function `robot()` and assign it to a variable `myrobot`
- now we can do things with `myrobot`
  - we can set a position `myrobot.set(10.0, 10.0, 0.0)`
    - these 3 values are the X coordinate, the Y coordinate, and the heading in radians, and this command assigns those values to the robot
  - we can make the robot move `myrobot.move(0.0, 10.0)`
    - this robot moves 10 meters forward and doesn't turn
  - the last thing I want to show you is how to generate measurements
    - `myrobot.sense()` gives you the distance to the 4 landmarks


- this code has a little bit more stuff
- it actually assimilates noise, but the noise filters are all set to $0$ and those noise filters are really important for particle filters so you can play with those if you like
  - in fact, there's a function over here called `set_noise()` which allows you to set them
- later on we have a function that makes kind of no sense right now, but really important as we implement particle filters called `measurement_prob()` which accepts a measurement and tells you how plausible this measurement is
  - it's kind of the key thing for the survival of the fittest rule in Particle Filters

# Moving Robot

- here's our first programming exercise
- I'd like you to make a robot that starts at coordinates $30$ and $50$, and it heads north, which means its heading direction is $\pi/2$
- it then turns clockwise by $\pi/2$, which means you subtract $\pi/2$ from the heading direction, and it moves $15$ meters
- it then senses, and I want you to print out the sensor measurements
- it then turns clockwise by $\pi/2$ again, and moves $10$ meters this time and I just want you to print out the sensor measurements after this entire procedure

In [2]:
from math import *
import random


landmarks = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0
        self.turn_noise = 0.0
        self.sense_noise = 0.0

    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError("X coordinate out of bound")
        if new_y < 0 or new_y >= world_size:
            raise ValueError("Y coordinate out of bound")
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError("Orientation must be in [0..2pi]")
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)

    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise)
        self.turn_noise = float(new_t_noise)
        self.sense_noise = float(new_s_noise)

    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt(
                (self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2
            )
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z

    def move(self, turn, forward):
        if forward < 0:
            raise ValueError("Robot cant move backwards")

        # turn, and add randomness to the turning command
        orientation = (
            self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        )
        orientation %= 2 * pi

        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size  # cyclic truncate
        y %= world_size

        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res

    def Gaussian(self, mu, sigma, x):

        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(-((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(
            2.0 * pi * (sigma ** 2)
        )

    def measurement_prob(self, measurement):

        # calculates how likely a measurement should be

        prob = 1.0
        for i in range(len(landmarks)):
            dist = sqrt(
                (self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2
            )
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob

    def __repr__(self):
        return "[x=%.6s y=%.6s orient=%.6s]" % (
            str(self.x),
            str(self.y),
            str(self.orientation),
        )


def eval(r, p):
    sum = 0.0
    for i in range(len(p)):  # calculate mean error
        dx = (p[i].x - r.x + (world_size / 2.0)) % world_size - (world_size / 2.0)
        dy = (p[i].y - r.y + (world_size / 2.0)) % world_size - (world_size / 2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

<IPython.core.display.Javascript object>

In [3]:
# Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again.
#
# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

myrobot = robot()

myrobot.set(30.0, 50.0, pi / 2)
myrobot = myrobot.move(-pi / 2, 15.0)
print(myrobot.sense())
myrobot = myrobot.move(-pi / 2, 10.0)
print(myrobot.sense())

[39.05124837953327, 46.09772228646444, 39.05124837953327, 46.09772228646444]
[32.01562118716424, 53.150729063673246, 47.16990566028302, 40.311288741492746]


<IPython.core.display.Javascript object>

# Add Noise

- next, I'd like you to play with the noise
- our class `robot` has built-in noise variables
  - one is for forward motion--this is the added Gaussian noise variable to the motion you command
  - the same for turn and the same for the sensor measurements
- I want you to--into your code--set these values as follows: `forward_noise` equals $5.0$, `turn_noise` equals $0.1$, and `sense_noise` equals $5.0$

In [4]:
####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

myrobot = robot()
# enter code here
forward_noise = 5.0
turn_noise = 0.1
sense_noise = 5.0
myrobot.set_noise(forward_noise, turn_noise, sense_noise)
myrobot.set(30.0, 50.0, pi / 2)
myrobot = myrobot.move(-pi / 2, 15.0)
print(myrobot.sense())
myrobot = myrobot.move(-pi / 2, 10.0)
print(myrobot.sense())

[45.35578159518798, 56.52837760277248, 35.04567917592496, 48.83381941909112]
[31.74325639705242, 53.35489027139334, 48.140109539986796, 47.78216420935595]


<IPython.core.display.Javascript object>

# Robot World

- now we know about our class robot who can turn and then move straight after the turn, and which it also can sense the distance to 4 designated landmarks, L1, L2, L3, and L4--these distances comprise the measurement vector of the robot
- we told you the robot lives in a world of size $100 x 100$, and what this means if this robot falls off one end, it disappears on the other--so, it's a cyclic world

# Creating Particles

- the particle filter that you're going to program maintains a set of $1000$ random guesses as to where the robot might be
- I'm not going to draw $1000$ dots here, but let me explain how each of these dots looks like
  - each of these dots is a vector which contains an $X$ coordinate, in this case $38.2$, a $Y$ coordinate $12.4$, and a heading direction of $0.1$, which is the angle at which the robot points relative to the $X$ axis
  - so, this robot moves forward, it will move slightly upwards
  - in fact, now a code--every time you call the function `robot()` and assign it say to a particle, here the $i^{th}$ particle, these elements `p[i].x`, `p[i].y`, and `p[i].orientation`, which is the same as heading, are initialized at random


- so, to make a particle set of $1000$ particles what you have to program is a simple piece of code that assigns $1000$ of those to a list

<img src="resources/creating_particles.png"/>

In [5]:
# Now we want to create particles,
# p[i] = robot(). In this assignment, write
# code that will assign 1000 such particles
# to a list.
#
# Your program should print out the length
# of your list (don't cheat by making an
# arbitrary list of 1000 elements!)
#
# Don't modify the code below. Please enter
# your code at the bottom.


# myrobot = robot()
# myrobot.set_noise(5.0, 0.1, 5.0)
# myrobot.set(30.0, 50.0, pi/2)
# myrobot = myrobot.move(-pi/2, 15.0)
# print myrobot.sense()
# myrobot = myrobot.move(-pi/2, 10.0)
# print myrobot.sense()

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

N = 1000
p = []

# enter code here
for i in range(N):
    p.append(robot())

print(len(p))
# print(p)

1000


<IPython.core.display.Javascript object>

- we now have a set of $1000$ particles, each of which just looks like one of these dots over here, and each of which has exactly $3$ values associated: an $X$, a $Y$, and an orientation

# Robot Particles

- I now want you to take each of these particles and simulate robot motion
- depending on the heading direction, this might yield a different direction
- so, each of these particles shall first turn by $0.1$ and then move by $5$ meters
  - we already implemented something just like this for individual robot motion and now I'd like you to apply this to the entire particle set

In [6]:
# Now we want to simulate robot
# motion with our particles.
# Each particle should turn by 0.1
# and then move by 5.
#
#
# Don't modify the code below. Please enter
# your code at the bottom.


# myrobot = robot()
# myrobot.set_noise(5.0, 0.1, 5.0)
# myrobot.set(30.0, 50.0, pi/2)
# myrobot = myrobot.move(-pi/2, 15.0)
# print myrobot.sense()
# myrobot = myrobot.move(-pi/2, 10.0)
# print myrobot.sense()

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

N = 1000
p = []

# enter code here
for i in range(N):
    p.append(robot())

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))

p = p2
print(len(p))
# print(p)

1000


<IPython.core.display.Javascript object>

- we've done the full recursion of applying our motion update to our full particle set
- if you've gotten this far then you got about half of particle filters implemented--unfortunately it's the easy half, but the difficult half isn't that much more difficult

# Importance Weight

<img src="resources/importance_weight_1.png"/>

- suppose an actual robot sits over here (blue, center), and it measures these exact distances to the landmarks over here
  - obviously, there some measurement noise that we just model as an added Gaussian with $0$ mean--meaning there would be a certain chance of being too short or too long and that probability is governed by a Gaussian
  - so, this process gives us a measurement vector of 4 values--of those 4 distances to the landmarks L1 to L4
- now let's consider a particle that hypothesizes the robot coordinates are over here (red) and not over here (blue, center), and it also hypothesizes a different heading direction
- we can then take the measurement vector and apply it to this particle (red)
  - obviously this would be a very poor measurement vector (black) for this specific particle
  - in particular, the measurement vector we would've expected looks more like this (green)
  - that just makes this specific location really unlikely
- the closer our particle to the correct position the more likely will be the set of measurements given that position

- now here comes the big trick in particle filters
- the mismatch of the actual measurement and the predicted measurement leads to a so called *importance weight* that tells us how important that specific particle is
  - the larger the weight the more important it is

<img src="resources/importance_weight_2.png"/>

- when we have many, many different particles and a specific measurement, each of these particles will have a different weight
  - some look very plausible, others might look very implausible as indicated by the size of the circles over here
- we now let these particles survive somewhat at random, but the probability of survival will be proportional to their weights
  - if something has a very big weight, it will survive at a higher proportion than something with a really small weight
- after what's called resampling--which is just a technical term for randomly drawing N new particles from old ones with replacement in proportion to the importance weight--after that resampling phase, big particles with big weights are very likely to live on, in fact many, many times whereas particles with small weights likely have died out


- the particles that are very consistent with the sensor measurement survive with a higher probability, and the ones with lower importance weight tend to die out
- we get the fact that the particles cluster around regions of higher posterior probability

- we have to implement a method for setting importance weights and that is, of course, related to the likelihood of a measurement
- and we have to implement a method for resampling that grabs particles in proportion to those weights


- I want you to program a way to assign importance weights to each of the particles
- I want you to make a list of 1000 elements where each element on the list contains a number which is proportional to how important that particle is
  - to make things easier I coded for you a function in the class `robot` called the `measurement_prob()`
  - this function accepts a single parameter, the measurement vector--the `Z` I just defined, and it calculates as an output how likely this measurement is, and it uses effectively a Gaussian that measures how far away the predicted measurements would be from the actual measurements


- we have to actually assume that there is measurement noise
  - if there is $0$ measurement noise, then this function will end up dividing by $0$--so let's put in a clause that specifies what we believe the actual measurement noise is
  - I'm not going to do this for the robot, but for the particles--$0.05$ for the translational noise, $0.05$ for the rotational noise, and $5.0$ for the measurement noise in those ranges


- I wish you to construct a list of $1000$ elements in `W` so that each number in this vector reflects the output of the function `measurement_prob()` applied to the measurement `Z` that we receive from the real robot, such that when I print `W`, I actually get a list of 1000 importance weights

In [7]:
# Now we want to give weight to our
# particles. This program will print a
# list of 1000 particle weights.
#
# Don't modify the code below. Please enter
# your code at the bottom.


# myrobot = robot()
# myrobot.set_noise(5.0, 0.1, 5.0)
# myrobot.set(30.0, 50.0, pi/2)
# myrobot = myrobot.move(-pi/2, 15.0)
# print myrobot.sense()
# myrobot = myrobot.move(-pi/2, 10.0)
# print myrobot.sense()

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
p = p2

w = []
# insert code here!
for i in range(N):
    w.append(p[i].measurement_prob(Z))

print(w)

[5.2689444789469005e-20, 1.6303675791808847e-58, 5.1794867852641534e-54, 1.4547573688508373e-72, 1.0524897363637117e-62, 1.5981891955804665e-111, 4.089011191022423e-32, 2.5024119024693427e-103, 2.8726509568496426e-15, 1.5923745322356782e-15, 9.264607669475663e-67, 2.9370247445205188e-52, 1.4845563636460418e-88, 5.7503542578603136e-64, 3.442157529442313e-59, 2.8540289900981125e-92, 1.3090745707712471e-30, 1.1741475000720252e-87, 1.2405219406185469e-18, 1.2947441470947322e-21, 2.7119861655104644e-79, 7.428592058925124e-06, 1.1894214208728232e-06, 4.841122981249009e-14, 1.9409864756083558e-104, 9.79384034199656e-75, 2.3349100103413496e-11, 5.999688122562903e-53, 5.707383487944128e-83, 4.283403038936981e-102, 5.313110399528355e-114, 1.303526816286325e-90, 1.2085516346386864e-13, 3.609209258227343e-84, 1.7332614817312015e-91, 9.031705002760128e-84, 1.5618264075502324e-112, 9.65113633597183e-61, 2.1535055658444308e-80, 2.3292981897868946e-75, 3.5860043094694664e-22, 6.151402293284513e-71, 9.

<IPython.core.display.Javascript object>

- some of them are more likely--the ones that are closer to the truth like $-5$--those are the particles that should really survive,
- those ones over here with a $-126$, those are really ready to die off because they are so far away from the truth we really don't need them anymore


- in the final step of the particle filter algorithm, we just have to sample particles from `P` with a probability that is proportional to its corresponding `W` value
- particles in `P` that have a large value should be drawn more frequently than the ones with a small value

# Resampling

- let me emphasize what resampling actually means
- we are given $N$ particles, each of which has $3$ values
- they also now have weights--these are simple floats or continuous values
- let's call big $W$ the sum of all these weights, and let's normalize them just for the consideration of what to do--it's called the normalized weights $\alpha$--so $\alpha_1$ would be the weight $1$ divided by the normalizer $W$, and so on all the way to $\alpha_N$
  - obviously it goes without saying that the sum of all alphas is now $1$, since we normalized them


- what resampling now does is it puts all these particles and then normalized weights into a big bag, and then it draws with replacement $N$ new particles by picking each particle with probability $\alpha$
  - for example, $\alpha_2$ might be large so we're going to pick this one, $p_2$, $\alpha_3$ might also be large so we pick that one, $\alpha_4$ might be really small but just by chance you might actually pick it and then we might pick $\alpha_2$ again
  - so, you get 2 versions of $p_2$, perhaps even $3$ versions of $p_2$, depending on the probabilities
- we have $N$ particles over here, we do this thing $N$ times, which is why I said with replacement we can draw multiple copies of the same particle, and in the end those particles that have a high-normalized weight alpha over here will occur likely--more frequently, in the new set over here
  - that's called resampling

<img src="resources/resampling.png"/>

# Never Sampled

- **Q:** Suppose we have 5 particles with the following importance weights: $0.6, 1.2, 2.4, 0.6, 1.2$. If I, in the process of resampling, randomly draw a particle in accordance to the normalized importance weights, what is the probability of drawing $p_1, p_2, p_3, p_4, p_5$?
- **A:** $0.1, 0.2, 0.4, 0.1, 0.2$ We just have to normalize those importance weights--the sum of those numbers is 6. We divide each weight by 6 to get matching probability. Obviously they add up to $1$.


- **Q:** Let me make these alpha values explicit and ask another question. Is it possible that $p_1$ is never sampled in the resampling step? Yes or No?
- **A:** The answer is yes--in the random resampling process something with an importance weight of $0.1$ is actually quite unlikely to be sampled into the next data set.


- **Q:** Let me now ask the same question about $p_3$, which is the particle with the largest importance weight. Is it possible that $p_3$ is never sampled in the resampling step?
- **A:** The answer is yes, again. Even though this importance weight over here is large, it could happen that in each of the $5$ resampling steps we pick one of the other $4$.


- **Q:** What is the probability of never sampling $p_3$? To answer this question assume we make a new particle set with $N=5$ new particles where particles are drawn independently and with replacement.
- **A:** The answer is $0.0777$ approximately. The way to obtain this is for this particle to never to be drawn in the resampling phase, we always have to draw $1$ of the remaining $4$ particles. Those together have a probability of $0.6$ to be drawn, which contrasts to the $0.4$ for $p_3$. So for $5$ independent samplings to draw $1$ of those $4$, we get a total probability of $0.6^5$, which is approximately $0.0777$. Put differently, there is about a $7.7\% $ chance that particle 3 over here is missing, but with almost $93\%$ probability we'd have this particle included. Put differently, the particles with small importance weights will survive at a much lower rate than the ones with larger importance weights, which is exactly what we wish to get from the resampling step.

# New Particle

- what I would like you to do next is to modify our algorithm to take the lists of particles and importance weights to sample $N$ times with replacement and new particles with a sampling probability proportional to the importance weights
- in the code now that we calculated our new particles and the corresponding importance weights, construct a new particle set `p3`, which we will call `p`, again, when everything is said and done, so that the particles in `p3` are drawn from `p` according to the importance weights `w`


- it turns out this is not an easy thing to do
- an obvious idea might be to compute all this normalized alphas, but you still have to be able to sample from those
- so, in the spectrum of our alphas you might draw a random variable uniformly from the interval $0;1$, and then find out the alpha so that all the alphas leading up to it, and some are smaller than beta but if we add the new alpha to the sum you would get a value larger than beta
  - that's doable but it's inefficient--in the best case you get an $N * log(N)$ implementation

# Resampling Wheel

<img src="resources/resampling_wheel.png"/>

- here's an idea how to make this more efficient, and it turns out empirically it also gives better samples
- let's represent all our particles and importance weight in a big wheel
- each particle occupies a slice that corresponds to its importance weight
  - particles with a bigger weight occupy more space whereas particles with a smaller weight occupy less space


- very initially let's guess a particle index uniformly from the set of all indices
  - I denote this as a uniform sample at $U$ from the discrete set of choices of index $1$ all the way to $N$, and as a caveat in Python, of course, it goes from $0$ to $N-1$
- say we pick $w_6$--then, the trick is--I'm going to construct the function $\beta$ and initialize to $0$ and to which I add--when I construct these particles--a uniformly drawn continuous value that sits between $0$ and $2 \times w_{max}$, which is the largest of the importance weights in the important set
  - $w_5$ is the largest, so we're going to add a random value that might be as large as twice $w_5$
- suppose the value we added brings us to here (green)--so, this is the value we actually drew, measured from the beginning of the sixth particle which shows an initialization
- I now then iterate the following loop:
  - if the importance weight of the present particle doesn't suffice to reach all the way to $\beta$--so, if $w_{index}$ isn't as big as $\beta$, then I subtract from $\beta$ this very value $w_{index}$ and I increment index by $1$
- we now get to the point where $\beta$ becomes smaller than $w_{index}$, which is the case in the next situation--now index=$7$
- then, index is the index of the particle I pick in my resampling process
- so, I picked the particle index; I now iterate I add another uniform value to beta(purple)
  - the same iteration now will make index flow up reducing beta by all the slice over here, which is $w_7$, and then jump over here, and particle $1$ is picked


- it can easily happen that the uniform value is so small that the same particle is picked twice, and it's easy to see that each particle is now picked in proportion to the total circumference it spans in this wheel of particles

```python
while w[index] < beta:
    beta = beta - w[index]
    index = index + 1

select p[index]
```

In [8]:
# In this exercise, try to write a program that
# will resample particles according to their weights.
# Particles with higher weights should be sampled
# more frequently (in proportion to their weight).

# Don't modify anything below. Please scroll to the
# bottom to enter your code.


# myrobot = robot()
# myrobot.set_noise(5.0, 0.1, 5.0)
# myrobot.set(30.0, 50.0, pi/2)
# myrobot = myrobot.move(-pi/2, 15.0)
# print myrobot.sense()
# myrobot = myrobot.move(-pi/2, 10.0)
# print myrobot.sense()

myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
p = p2

w = []
for i in range(N):
    w.append(p[i].measurement_prob(Z))


#### DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
# You should make sure that p3 contains a list with particles
# resampled according to their weights.
# Also, DO NOT MODIFY p.

p3 = []
index = int(random.random() * N)
beta = 0.0
max_w = max(w)
for i in range(N):
    beta += random.random() * 2 * max_w
    while beta > w[index]:
        beta -= w[index]
        index = (index + 1) % N
    p3.append(p[index])

p = p3
print(p)

[[x=15.706 y=8.0736 orient=3.0710], [x=15.706 y=8.0736 orient=3.0710], [x=15.095 y=11.352 orient=1.7743], [x=15.095 y=11.352 orient=1.7743], [x=16.530 y=2.3656 orient=0.5950], [x=11.079 y=9.7569 orient=4.0061], [x=8.2609 y=9.6771 orient=4.3370], [x=13.477 y=10.896 orient=0.5868], [x=13.013 y=6.1219 orient=3.5685], [x=6.8378 y=12.791 orient=2.2780], [x=16.139 y=10.876 orient=0.6076], [x=25.778 y=2.5858 orient=6.2300], [x=15.706 y=8.0736 orient=3.0710], [x=15.095 y=11.352 orient=1.7743], [x=18.026 y=10.423 orient=5.6052], [x=11.712 y=9.0370 orient=4.4482], [x=15.644 y=4.2265 orient=0.3405], [x=19.437 y=8.3928 orient=3.1099], [x=13.013 y=6.1219 orient=3.5685], [x=13.013 y=6.1219 orient=3.5685], [x=18.967 y=9.3133 orient=4.5376], [x=15.706 y=8.0736 orient=3.0710], [x=15.706 y=8.0736 orient=3.0710], [x=15.095 y=11.352 orient=1.7743], [x=11.079 y=9.7569 orient=4.0061], [x=11.079 y=9.7569 orient=4.0061], [x=11.712 y=9.0370 orient=4.4482], [x=8.2609 y=9.6771 orient=4.3370], [x=13.477 y=10.896 

<IPython.core.display.Javascript object>

# Orientation

- **Q:** Will orientation or heading never play a role? And the answers are: Yes, Never--so we always get a random set of orientations or No--eventually they matter?
- **A:** The correct answer is No--of course they will eventually matter. Orientation does matter in the second step of particle filtering because the prediction is so different for different orientations.

- I want you to take this particle filter, and program it to run twice

In [9]:
# myrobot = robot()
# myrobot.set_noise(5.0, 0.1, 5.0)
# myrobot.set(30.0, 50.0, pi/2)
# myrobot = myrobot.move(-pi/2, 15.0)
# print myrobot.sense()
# myrobot = myrobot.move(-pi/2, 10.0)
# print myrobot.sense()

####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER/MODIFY CODE BELOW ####
myrobot = robot()
N = 1000
T = 2

p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

for t in range(T):
    myrobot = myrobot.move(0.1, 5.0)
    Z = myrobot.sense()

    p2 = []
    for i in range(N):
        p2.append(p[i].move(0.1, 5.0))
    p = p2

    w = []
    for i in range(N):
        w.append(p[i].measurement_prob(Z))

    p3 = []
    index = int(random.random() * N)
    beta = 0.0
    mw = max(w)
    for i in range(N):
        beta += random.random() * 2.0 * mw
        while beta > w[index]:
            beta -= w[index]
            index = (index + 1) % N
        p3.append(p[index])
    p = p3

print(p)  # Leave this print statement for grading purposes!

[[x=34.208 y=33.873 orient=4.1947], [x=34.178 y=33.807 orient=4.1961], [x=34.032 y=33.828 orient=4.1691], [x=40.673 y=36.827 orient=4.8469], [x=36.533 y=38.653 orient=2.7198], [x=40.466 y=36.813 orient=4.8058], [x=36.839 y=40.283 orient=0.1947], [x=36.696 y=39.137 orient=2.6190], [x=36.569 y=38.895 orient=2.6732], [x=33.549 y=34.378 orient=4.0270], [x=36.683 y=38.992 orient=2.6455], [x=38.920 y=34.057 orient=4.1554], [x=36.976 y=39.423 orient=4.9440], [x=36.580 y=38.628 orient=2.7206], [x=31.537 y=43.440 orient=2.6891], [x=40.205 y=32.549 orient=4.4133], [x=37.128 y=37.620 orient=4.8513], [x=33.856 y=34.050 orient=4.1161], [x=36.751 y=39.003 orient=2.6369], [x=40.542 y=36.815 orient=4.8207], [x=34.009 y=33.924 orient=4.1553], [x=34.009 y=33.924 orient=4.1553], [x=36.558 y=38.899 orient=2.6735], [x=37.564 y=37.831 orient=4.9446], [x=36.918 y=40.121 orient=0.1603], [x=33.723 y=34.148 orient=4.0830], [x=33.892 y=34.064 orient=4.1205], [x=33.892 y=34.064 orient=4.1205], [x=41.055 y=36.907 

<IPython.core.display.Javascript object>

# Error

- rather than printing out the particles themselves, I want you to print out the overall quality of the solution
    - to do this, I've programmed for you an `eval` code which takes in as a robot position and a particle set, and it computes the average error of each particle, relative to the robot pose in $X$ and $Y$--not in the orientation
  - the way it does it, it basically compares the $X$ of the particle with the $X$ of the robot, computes the Euclidian distance of these differences, and averages all of those--so it sums them all up and it averages them, through the number of particles over here, which is upper caps $N$
  - there's some funny stuff over here--the reason is, the world is cyclic and it might be that the robot is at $0.0$ or at $99.9$--it's about the same values but, according to the calculation, they'd be different
    - so while there's normalization over here, I make sure that the cyclicity of the world doesn't really affect negatively the estimated error I've enclosed in the boundary

In [10]:
####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER/MODIFY CODE BELOW ####
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()
N = 1000
T = 10  # Leave this as 10 for grading purposes.

p = []
for i in range(N):
    r = robot()
    r.set_noise(0.05, 0.05, 5.0)
    p.append(r)

for t in range(T):
    myrobot = myrobot.move(0.1, 5.0)
    Z = myrobot.sense()

    p2 = []
    for i in range(N):
        p2.append(p[i].move(0.1, 5.0))
    p = p2

    w = []
    for i in range(N):
        w.append(p[i].measurement_prob(Z))

    p3 = []
    index = int(random.random() * N)
    beta = 0.0
    mw = max(w)
    for i in range(N):
        beta += random.random() * 2.0 * mw
        while beta > w[index]:
            beta -= w[index]
            index = (index + 1) % N
        p3.append(p[index])
    p = p3

    # enter code here, make sure that you output 10 print statements.
    print(eval(myrobot, p))

5.973190518296271
7.903663287857302
9.46430846786852
11.05035709058652
12.229847010230959
12.099732128503387
11.099935665313167
10.52578099729524
10.050858537763181
9.560926980510569


<IPython.core.display.Javascript object>

# Conclusion

- you've just programmed a full Particle Filter
  - you got, from me, kind of a very primitive and rudimentary robot simulator that uses landmarks as a way of doing measurements, and it uses three-dimensional robot coordinates, so it's a little bit better than the stuff we talked about in previous classes--it's not quite a Google car yet
  - you solved the estimation problem--in fact, you solved this problem in 30 lines of code

- we had measurement updates and motion updates
  - in the measurement update, we computed posterior over state given the measurement and it was proportional to, after normalization of probability of the measurement given the state times $p$ of the state itself
  - in the motion update, if you compute a posterior of the distribution one time step later and that is the convolution of the transition probability times my prior

<img src="resources/measurement_and_motion_updates_formulas.png"/>
  
- those formulas--those should look familiar--this is exactly what you implemented.
  - for measurement update
    - $P(X)$ distribution was a set of particles--a thousand particles together represented your prior $X$
    - $P(Z|X)$ were importance weights
    - technically speaking, the particles with the importance weights are a representation of distribution but we wanted to get rid of the importance weights
    - so by resampling, we work the importance weights back into the set of particle so the resulting particles--$P(X|Z)$--would represent the correct posterior
  - for motion updates
    - $P(X)$ was your set of particles again
    - you sampled from the sum by taking a random particle and applying the motion model with a noise model to generate a random particle, $X'$
    - as a result, you get a new particle set that is the correct distribution after the robot motion